In [19]:
vocab_size = 15165
import numpy as np
import pandas as pd
import re
import random
import pickle
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
class WordLSTM(nn.Module):
    def __init__(self,n_hidden=256,n_layers = 4,drop_prob = 0.3,lr = 0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        self.emb_layer = nn.Embedding(vocab_size,200)
        self.lstm = nn.LSTM(200,n_hidden,n_layers,dropout = drop_prob,batch_first = True)
        self.dropout = nn.Dropout(drop_prob)
        self.fc = nn.Linear(n_hidden,vocab_size)
    
    def forward(self,x,hidden):
        embedded = self.emb_layer(x)
        lstm_output,hidden = self.lstm(embedded,hidden)
        out = self.dropout(lstm_output)
        out = out.reshape(-1,self.n_hidden)
        out = self.fc(out)
        return out,hidden
    
    def init_hidden(self,batch_size):
        weight = next(self.parameters()).data

        if (torch.cuda.is_available()):
            hidden = (weight.new(self.n_layers,batch_size,self.n_hidden).zero_().to(device),
                      weight.new(self.n_layers,batch_size,self.n_hidden).zero_().to(device))
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        return hidden

In [4]:
model = torch.load('model.pt',map_location=torch.device('cpu'))
print(model.eval())

WordLSTM(
  (emb_layer): Embedding(15165, 200)
  (lstm): LSTM(200, 256, num_layers=4, batch_first=True, dropout=0.3)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc): Linear(in_features=256, out_features=15165, bias=True)
)


In [6]:
def predict(model, tkn, h=None):
         
  # tensor inputs
  x = np.array([[token2int[tkn]]])
  inputs = torch.from_numpy(x)
  
  # push to GPU
  inputs = inputs.to(device)

  # detach hidden state from history
  h = tuple([each.data for each in h])

  # get the output of the model
  out, h = model(inputs, h)

  # get the token probabilities
  p = F.softmax(out, dim=1).data

  p = p.cpu()

  p = p.numpy()
  p = p.reshape(p.shape[1],)

  # get indices of top 3 values
  top_n_idx = p.argsort()[-3:][::-1]

  # randomly select one of the three indices
  sampled_token_index = top_n_idx[random.sample([0,1,2],1)[0]]

  # return the encoded value of the predicted char and the hidden state
  return int2token[sampled_token_index], h

In [5]:
def generate_text(model,size,prime = "once upon"):
    model.to(device)
    model.eval()
    # batch size is 1
    h = model.init_hidden(1)
    toks = prime.split()

    for t in prime.split():
        token,h = predict(model,t,h)
    toks.append(token)

    for i in range(size-1):
        token,h = predict(model,toks[-1],h)
        toks.append(token)
    return " ".join(toks)

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [15]:
import re
import pickle
file = open('/content/plots_text.pickle','rb')
movie_plots = pickle.load(file)
movie_plots = [re.sub("[^a-z]"," ",i)for i in movie_plots]
# creating integer-to-token mapping
int2token = {}
count = 0
for i in set(" ".join(movie_plots).split()):
    int2token[count] = i
    count += 1

# creating token-to-integer mapping

token2int = {t: i for i,t in int2token.items()}

In [20]:
generate_text(model,15,prime='once i was')

'once i was framed wilde round chickens bitterness transit pitcher notified transit pitcher notified round extent drug while'